In [6]:
import joblib
from joblib import Parallel,delayed
import pandas as pd
import numpy as np
import datetime as dt
import os
from tqdm import tqdm

import seaborn as sns
import fxcmpy
import sys
import time
sys.path.append('../') 
from librairies.strategy import *
from librairies.dagfeaturingfx import *
import pyttsx3
engine = pyttsx3.init()

In [7]:
_token = 'dbdc379ce7761772c662c3e92250a0ae38385b2c'
_server = 'demo'
_user_id = 'D261282181'
_compte = '01215060'
_password = 'waXz1'

_period = 'm15'
_name = 'MLPClassifier'

In [9]:
%%time

def conX():
    con = fxcmpy.fxcmpy(access_token=_token, log_level='error',server=_server)
    if con.is_connected() == True:
        print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
    else:
        print(col.Fore.RED+'Connexion non établie'+col.Style.RESET_ALL)
    return(con)

def deconX():
    con = con.close()
    if con.is_connected() == True:
        print(col.Fore.GREEN+'Connexion non intérrompue'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
    else:
        print(col.Fore.RED+'Connexion intérrompue'+col.Style.RESET_ALL)
    return()

def scrap_hist(ticker,invers = 'non'):
    #_debut = pd.to_datetime((dt.datetime.now()-dt.timedelta(minutes=3987165)).strftime('%Y-%m-%d'))
    #_fin = pd.to_datetime((dt.datetime.now().strftime('%Y-%m-%d')))
    data = con.get_candles(ticker,period=_period,start=_debut,end=_fin)
    data['Open'] = (data['bidopen']+data['askopen'])/2
    data['High'] = (data['bidhigh']+data['askhigh'])/2
    data['Low'] = (data['bidlow']+data['asklow'])/2
    data['Close'] = (data['bidclose']+data['askclose'])/2
    return(data)


try:
    con.is_connected() == True
    
    engine.say("already Connected")
    engine.runAndWait()
    print(col.Fore.GREEN+'Connexion rétablie'+col.Style.RESET_ALL)
    print('Compte utilisé : ',con.get_account_ids())
    print('')
    
except:
    try:
        con = conX()
        con.is_connected() == True
        print(col.Fore.GREEN+'Connexion établie'+col.Style.RESET_ALL)
        print('Compte utilisé : ',con.get_account_ids())
        engine.say("Bigfoot is Connected")
        engine.runAndWait()
    except:
        print(col.Fore.RED+'Connexion non établie'+col.Style.RESET_ALL)
        engine.say("Mayday, mayday, Not Connected, mauzerfucker!")
        engine.say("Check your internet, and launch agin the Bigfoot")
        engine.runAndWait()
        print('')
        #os._exit(0)
        con = deconX()
        time.sleep(1)
        con = conX()

TICKER_LIST = list(sorted(set(get_ticker_list())))


for x in tqdm(TICKER_LIST):
    _path = 'JOBLIB/Ticker_'+_period+'/df_'+x.replace('/','')
    df_all = joblib.load('JOBLIB/Built_bases/df_all')
    df_all = df_all[df_all.Symbol==x.replace('/','')]
    df_all = df_all[['Close','CloseAsk','CloseBid','High','HighAsk','HighBid','Low','LowAsk','LowBid','Open','OpenAsk','OpenBid','Symbol','Date']]
    _fin = dt.datetime.now()
    _deb = df_all.index[-1]
    _debut = dt.datetime(_deb.year,_deb.month,_deb.day,_deb.hour,_deb.minute)
    addon = con.get_candles(x,period='m15',start=_debut,end=_fin).drop(['tickqty'],axis=1)
    addon = addon.rename(columns={'bidopen':'OpenBid','bidclose':'CloseBid','bidhigh':'HighBid','bidlow':'LowBid','askopen':'OpenAsk','askclose':'CloseAsk','askhigh':'HighAsk','asklow':'LowAsk'})
    addon['Open'] = (addon.OpenAsk + addon.OpenBid)/2
    addon['High'] = (addon.HighAsk + addon.HighBid)/2
    addon['Low'] = (addon.LowAsk + addon.LowBid)/2
    addon['Close'] = (addon.CloseAsk + addon.CloseBid)/2
    addon['Symbol'] = x.replace('/','')
    addon['Date'] = addon.index
    addon['Date'] = pd.to_datetime(addon['Date'].dt.strftime(date_format='%Y-%m-%d'))
    df_all = df_all.append(addon.iloc[1:,:])
    joblib.dump(df_all,_path)


  0%|          | 0/42 [00:00<?, ?it/s]Connexion rétablie
Compte utilisé :  [1277536]


Récupération des tickers
100%|██████████| 42/42 [05:28<00:00,  7.83s/it]CPU times: user 2min 37s, sys: 2min 34s, total: 5min 12s
Wall time: 5min 31s

